In [482]:
import pymoo
import string
from pymoo.core.problem import ElementwiseProblem
import numpy as np

In [483]:
phrase = "To be, or not to be, that is the question"

In [484]:
phrase_list = [i for i in phrase]

In [485]:
global phrase_list

In [486]:
class ShakespeareProblem(ElementwiseProblem):
    def __init__(self, phrase_list):
        xl = np.zeros(len(phrase_list))

        xu = np.ones(len(phrase_list)) * 99

        super().__init__(n_var=len(phrase_list), n_obj=1, n_ieq_constr=2, xl=xl, xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):

        score = 0
        for i in range(len(phrase_list)):
            if phrase_list[i] == string.printable[int(x[i])]:
                score += 1

        out["F"] = -1 * score
        #out["G"] = -1 * score

In [487]:
ITERATIONS = 200
POPULATION = 1000

In [488]:
problem = ShakespeareProblem(phrase_list=phrase_list)

In [489]:
def decode_String(pop_individual):
    final_str = ""
    for i in pop_individual:
        final_str += (string.printable[int(i)])
    return final_str

In [490]:
from pymoo.util.display.column import Column
from pymoo.util.display.output import Output


class MyOutput(Output):

    def __init__(self):
        super().__init__()
        self.score = Column("score", width=13)
        self.Phrase = Column("Phrase", width=35)
        self.columns += [self.score, self.Phrase]

    def update(self, algorithm):
        super().update(algorithm)
        self.score.set(-np.min(algorithm.pop.get("F")))
        self.Phrase.set(decode_String(algorithm.pop.get("X")[0]))

In [491]:
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA

algorithm = GA(pop_size=POPULATION)

term = get_termination("n_gen", ITERATIONS)

res = minimize(problem,
               algorithm,
               #seed=SEED,
               save_history=True,
               output=MyOutput(),
               verbose=True,
               termination = term)

n_gen  |  n_eval  |     score     |                Phrase              
     1 |     1000 |  4.0000000000 | c	{"8eT!4EHlS~Q3K=&ln8)gE!ER`:1VoC

     2 |     2000 |  4.0000000000 | /orbO(4(l|a<~JCSZB7usl1THHiJam|E0q>
     3 |     3000 |  4.0000000000 | /orbO(4(l|a<~JCSZB7usl1THHiJam|E0q>
     4 |     4000 |  4.0000000000 | /orbO(4(l|a<~JCSZB7usl1THHiJam|E0q>
V!o@N46.ddv@9 U1:ha;.O%'xhP _g000 | H2L,
V!o@N46.ddv@9 U1:ha;.O%'xhP _g000 | H2L,
     7 |     7000 |  6.0000000000 | TSg5lP_Ez;4/g%to9M@UhPs"C BovtFObd1
     8 |     8000 |  6.0000000000 | TSg5lP_Ez;4/g%to9M@UhPs"C BovtFObd1
     9 |     9000 |  8.0000000000 | T] #Z| vB lXbn$oKcCd5[KA~yc(QtbeWqg
    10 |    10000 |  8.0000000000 | T] #Z| vB lXbn$oKcCd5[KA~yc(QtbeWqg
    11 |    11000 |  8.0000000000 | T] #Z| vB lXbn$oKcCd5[KA~yc(QtbeWqg
    12 |    12000 |  8.0000000000 | T] #Z| vB lXbn$oKcCd5[KA~yc(QtbeWqg
    13 |    13000 |  8.0000000000 | T] #Z| vB lXbn$oKcCd5[KA~yc(QtbeWqg
    14 |    14000 |  8.0000000000 | T] #Z| vB lXbn$oKc

In [492]:
decode_String(res.pop.get("X")[0])

'To be, or not to be, that is the question'

In [493]:
99 ** len(phrase)

6622820409839837087989552128156750893507779541456909263939129526255095863651804099

In [494]:
np.float_power(99, len(phrase))

6.622820409839837e+81